In [74]:
import gymnasium as gym
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

from typing import Optional
import gymnasium as gym

from typing import Optional, List, Dict

import numpy as np
import pandas as pd
import numpy as np
import torch

import const 

In [75]:
from typing import Callable

def make_env(env_id: str, rank: int, seed: int = 0) -> Callable:
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    :return: (Callable)
    """

    def _init() -> gym.Env:
        env = gym.make(env_id)
        env.reset(seed=seed + rank)
        return env

    set_random_seed(seed)
    return _init

In [76]:
# gym.Env를 상속해서 새로운 환경을 만들어야 한다.
class MarketEnv(gym.Env):
    """
    action space는 이용 가능한 portfolio n개에 대해서, 각각의 비율을 조절하는 것이다.
    해당 action을 취하면 결과로 취득할 수 있는 것이 differential sharpe ratio와 
    
    sharpe ratio: risk-adjusted return
    
    첫 거래일, 바로 다음 거래일, 최종 거래일
    
    """
    def __init__(self,
                 lookback_T: int,
                 sharpe_eta: float,
                 asset_definition: Dict[str, str],
                 market_df: pd.DataFrame,
                 debug: bool = False
                 ):
        
        self.debug = debug
        
        # some definiitions of assets
        self.market_df: pd.DataFrame = market_df
        self.idx2asset = {i: asset for i, asset in enumerate(asset_definition.keys())}
        self.idx2asset[len(asset_definition)] = 'cash'
        self.asset2idx = {asset: i for i, asset in enumerate(asset_definition.keys())}
        self.asset2idx['cash'] = len(asset_definition)

        self.lookback_T = lookback_T
        self.business_days = len(market_df)
        self.num_securities = len(asset_definition)
        self.num_all_asset = self.num_securities + 1  # including cash
        self.share_eta = sharpe_eta
        
        # RL agent 학습을 위한 state 정보
        
        # time stamp 만들기
        self.time_step = 0
        # [S_1, S_2, ..., S_n], 나중에 debug하기 쉽도록 전체 정보에 대한 저장
        self.overall_state = np.zeros(
            (self.business_days, self.num_all_asset, self.lookback_T)
        )  # to handle cash
        
        # portfolio 가치 산정을 위한 부분
        # 현금 이외의 금액과 현금 금액
        self.portfolio_ac = np.zeros((self.business_days, 2))
        # 실 거래를 위한 quantized shares
        self.portfolio_shares = np.zeros((self.business_days, self.num_securities))
        # sharpe ratio 계산을 위한 부분
        self.portfolio_return = np.zeros((self.business_days, 1))
        
        self.observation_space = gym.spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(self.num_all_asset, self.lookback_T),
            dtype=np.float32
        )
        
        self.action_space = gym.spaces.Box(
            low=0,
            high=1,
            shape=(self.num_securities,),
            dtype=np.float32
        )
        # n개의 자산에 대해서 예측을 하되, cash의 경우에는 1 - \sum_ i w_i로 계산
        
        ### example
        # # Define the agent and target location; randomly chosen in `reset` and updated in `step`
        # self._agent_location = np.array([-1, -1], dtype=np.int32)
        # self._target_location = np.array([-1, -1], dtype=np.int32)
        # # Observations are dictionaries with the agent's and the target's location.
        # # Each location is encoded as an element of {0, ..., `size`-1}^2
        # self.observation_space = gym.spaces.Dict(
        #     {
        #         "agent": gym.spaces.Box(0, size - 1, shape=(2,), dtype=int),
        #         "target": gym.spaces.Box(0, size - 1, shape=(2,), dtype=int),
        #     }
        # )
        # # We have 4 actions, corresponding to "right", "up", "left", "down"
        # self.action_space = gym.spaces.Discrete(4)
        # # Dictionary maps the abstract actions to the directions on the grid
        # self._action_to_direction = {
        #     0: np.array([1, 0]),  # right
        #     1: np.array([0, 1]),  # up
        #     2: np.array([-1, 0]),  # left
        #     3: np.array([0, -1]),  # down
        # }
        ###
    
    def _get_asset_columns(self, suffix: str):
        """
        cash를 제외한 column들을 가져오기 위함
        """
        asset_columns = [f"{asset}_{suffix}" for asset in self.idx2asset.values()]
        del asset_columns[-1]
        
        return asset_columns
        
        
    
    def reset(self, initial_cash: float, seed: Optional[int] = None, options: Optional[dict] = None):
        """
        여기서 overall obersevation를 잘 정의 해주도록 한다.
        """
        # We need the following line to seed self.np_random
        super().reset(seed=seed)
        
        # 처음 거래 시작일, portfolio의 처음은 당연하게도 현금만 운용 가능
        self.business_start_idx = self.time_step = self.lookback_T - 1
        
        # RL environment 초기화
        asset_columns = self._get_asset_columns("log_r")
        
        for t in range(self.lookback_T-1,self.business_days, 1):
            # 당일 전부터 lookback_T일 전까지의 데이터를 저장
            self.overall_state[t, :self.num_securities, 1:] = self.market_df[asset_columns][(t-(self.lookback_T-1)):t][::-1].to_numpy().T
            
            # 현금과 변동성 정보
            self.overall_state[t, -1][1:3] = self.market_df[["SPX_vol20_normalized", "SPX_vol20_div_vol60_normalized"]][t-1:t].to_numpy()
            self.overall_state[t, -1][3:] = self.market_df[["VIX_close_normalized"]][t-(self.lookback_T-3):t][::-1].to_numpy().T
        self.overall_state[:self.lookback_T, -1, 0] = 1.0  # cash

        # portfolio 초기화
        # T-1부터 거래를 시작해야함, 실제로는 T일에 해당하는 거래 data를 활용해야 함
        self.portfolio_ac[:self.lookback_T, 1] = initial_cash
        
        ### example
        # Choose the agent's location uniformly at random
        # self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)
        # # We will sample the target's location randomly until it does not coincide with the agent's location
        # self._target_location = self._agent_location
        # while np.array_equal(self._target_location, self._agent_location):
        #     self._target_location = self.np_random.integers(
        #         0, self.size, size=2, dtype=int
        #     )
        ### example

        observation = self._get_obs()
        info = self._get_info()

        return observation, info

    def _get_obs(self):
        # return {"agent": self._agent_location, "target": self._target_location}
        return {
            "state_t": self.overall_state[self.time_step],
        }
        # agent는 특정 시점일때의 정보에만 접근가능 하도록 하기
        
    
    def _get_info(self):
        """
        여기는 internal 분석용으로 쓰면 될 듯 하다
        여기서는 portfolio의 가치와 shares 반환하면 검산할 때 편할 듯
        """
        return {
            "current_step": self.time_step,
            "port_val_sum": self.portfolio_ac[self.time_step].sum(),
            "port_val": self.portfolio_ac[self.time_step],
            "port_shares": self.portfolio_shares[self.time_step],
        }
    
    def _compute_clip(self, action: np.ndarray):
        # action model이 예측한 raw weights
        
        columns = self._get_asset_columns("close")
        current_asset_prices = self.market_df[columns][self.time_step:self.time_step+1].to_numpy()
        current_securities_sum = np.sum(self.portfolio_shares[self.time_step-1] * current_asset_prices)
        current_portfolio_sum = current_securities_sum + self.portfolio_ac[self.time_step-1, 1]
        
        self.portfolio_shares[self.time_step, :] = np.floor((action * current_portfolio_sum) / current_asset_prices)
        
        self.portfolio_ac[self.time_step, 0] = np.sum(self.portfolio_shares[self.time_step, :] * current_asset_prices)  # 새로운 비중으로 계산된 securities
        self.portfolio_ac[self.time_step, 1] = current_cash = current_portfolio_sum - np.sum(self.portfolio_shares[self.time_step, :] * current_asset_prices)  # 새로운 비중으로 계산된 cash
        
        self.portfolio_return[self.time_step] = (self.portfolio_ac[self.time_step].sum() - self.portfolio_ac[self.time_step-1].sum()) / self.portfolio_ac[self.time_step-1].sum()
        
        # import ipdb; ipdb.set_trace()
        
        self.overall_state[self.time_step, :self.num_securities, 0] = (self.portfolio_shares[self.time_step, :] * current_asset_prices) / current_portfolio_sum
        self.overall_state[self.time_step, -1, 0] = current_cash / current_portfolio_sum
    
    def _compute_reward(self):
        """
        Differntial Sharpe Ratio 계산
        
        """
        # 거래 시작일 부터 portfolio의 return을 계산해야 함
        
        A_t1 = self.portfolio_return[self.business_start_idx:self.time_step].mean()
        delta_A_t = self.portfolio_return[self.time_step] - A_t1
        A_t = A_t1 + self.share_eta * delta_A_t

        B_t1 = (self.portfolio_ac[self.business_start_idx:self.time_step, 0] ** 2).mean()
        delta_B_t = (self.portfolio_return[self.time_step] ** 2) - (B_t1)
        B_t = B_t1 + self.share_eta * delta_B_t

        # import ipdb; ipdb.set_trace()
        if abs(B_t1 - A_t1**2) == const.epsilon: 
            diff_sharpe_ratio = 0.0
        else:
            diff_sharpe_ratio = ((B_t1 * delta_A_t - (1/2)*A_t1*delta_B_t)/(B_t1 - A_t1**2)**(3/2))
        
        return diff_sharpe_ratio
        
    def step(self, action):
        """
        action은 각 portfolio 배분 비율로 하면 될듯 하다.
        
        action을 취했을 때, 어떠한 결과를 얻어야 하는가?
        
        PortVal_t = \sum P_{i,t} \times shares_{i, t-1} + cash_{t-1}
        
        shares 
        """
        self.time_step += 1
        
        self._compute_clip(action)  # 자산 가치 계산
        
        reward = self._compute_reward()
        observation = self._get_obs()
        info = self._get_info()
        
        # An environment is completed if and only if the agent has reached the target
        terminated = True if self.time_step == self.business_days - 1 else False
        truncated = False

        # # Map the action (element of {0,1,2,3}) to the direction we walk in
        # direction = self._action_to_direction[action]
        # # We use `np.clip` to make sure we don't leave the grid bounds
        # self._agent_location = np.clip(
        #     self._agent_location + direction, 0, self.size - 1
        # )
        
        return observation, reward, terminated, truncated, info

In [77]:

# env snippet
"""
state를 초기화 하기 위한 함수

T-1 거래일 부터 정보들을 기입하기로 한다.
그 이전의 정보에 대해서는 접근할 수 없다고 생각한다.
"""
test_df = pd.read_csv('./temp/processed_data0610_test.csv')

test_env = MarketEnv(
    lookback_T=const.LOOKBACK_T,
    sharpe_eta=const.SHARPE_ETA,
    asset_definition=const.SNP_INDICES_ASSETS,
    market_df=test_df
)

init = test_env.reset(initial_cash=100_000)
test_action = np.ones(test_env.num_securities) / test_env.num_securities
test_action = np.zeros(test_env.num_securities)
test_action[5] = 1.0

In [78]:
init


({'state_t': array([[ 0.00000000e+00, -3.82191802e-03,  2.53750800e-04,
           1.14371693e-03, -1.52428755e-03, -1.26782801e-04,
          -2.27578415e-03,  1.51586360e-03,  1.77522079e-03,
          -3.66922502e-03,  3.16127820e-03,  5.62035997e-03,
           9.00892714e-04,  1.80741127e-03, -3.86385540e-03,
           1.28242872e-04, -2.30259764e-03, -2.92455302e-03,
          -2.27512181e-03, -5.03968124e-04,  8.77649409e-03,
          -3.83767871e-03,  4.35194104e-03,  1.28660787e-04,
          -2.69390440e-03,  3.98261301e-03, -5.51451753e-03,
          -4.56355669e-03,  4.56355669e-03,  1.02066905e-03,
           5.26891320e-03, -3.47694993e-03,  1.92820125e-03,
          -2.44094568e-03,  8.01638058e-03,  3.91667980e-04,
          -2.08482600e-03, -4.52596232e-03, -7.14484122e-03,
           2.41099090e-03, -3.29588754e-03,  1.89841982e-03,
           4.59035435e-03,  2.70028527e-03,  6.45407285e-04,
           9.05184598e-04, -1.16346262e-03, -1.27218962e-02,
           3.

In [79]:
step = test_env.step(test_action)

/tmp/ipykernel_3320522/2923400712.py:202: RuntimeWarning: invalid value encountered in divide
  diff_sharpe_ratio = ((B_t1 * delta_A_t - (1/2)*A_t1*delta_B_t)/(B_t1 - A_t1**2)**(3/2))


In [80]:
test_env.business_start_idx, test_env.time_step

(59, 60)

In [81]:
finished = False
while not finished:
    ret = test_env.step(test_action)
    finished = ret[2]
    if np.isnan(ret[0]["state_t"]).any():
        import ipdb; ipdb.set_trace()

ret, ret[1]

(({'state_t': array([[ 0.00000000e+00, -8.56174712e-04,  4.27876371e-04,
           -1.07008620e-04,  8.56808134e-04, -7.49799514e-04,
           -1.07008620e-04,  2.68315744e-03,  3.23095724e-04,
            5.39027600e-04,  9.71938821e-04, -2.37202144e-03,
            3.22695605e-04, -1.39661920e-03,  2.15050938e-03,
            0.00000000e+00,  3.78920685e-03,  0.00000000e+00,
            3.26332751e-04,  1.19865571e-03,  5.38002825e-03,
            9.60669968e-03, -6.05689105e-03, -2.55435414e-03,
           -1.98863249e-03,  7.56087319e-03, -6.34668018e-03,
            2.54978986e-03,  1.00183316e-03,  7.75847669e-03,
           -1.13437243e-04, -8.20189871e-03, -1.88799842e-03,
           -6.16186951e-03, -7.68874544e-03,  9.67129024e-04,
           -1.28902736e-03,  2.14572377e-04,  1.07325957e-04,
            6.70654868e-03,  2.07613810e-03,  3.96116284e-03,
            1.43934568e-03, -3.32580347e-04,  3.32580347e-04,
            1.77807610e-03,  6.68659842e-04,  1.11672595e-0

In [82]:
test_env.overall_state[test_env.time_step-1:, :, 0]
test_env.portfolio_ac[test_env.time_step-2:]

columns = test_env._get_asset_columns("close")
test_df.loc[test_env.time_step:test_env.time_step+1].to_numpy()


array([[1258, '2010-12-31', 1257.6, 404.5, 128.74, 239.61, 120.58,
        303.58, 214.77, 506.75, 364.78, 295.54, 301.12, 159.34, 17.75,
        -0.000238492725972, -0.000103588427898, -0.0022200296003946,
        -0.0009652184086987, 0.0031948881789137, 0.0013853105267664,
        0.0007935844958651, 0.0003445126855885, -0.0026468155500414,
        -0.0011510213314381, 0.0003954392671192, 0.0001717031447919,
        0.0020996640537514, 0.0009109165373439, -0.0004142338646046,
        -0.0001799367521368, -0.0012321003203462, -0.0005354242862709,
        -0.0023629489603024, -0.0010274300554917, 0.0011303943081322,
        0.0004906467504654, -6.275494195162162e-05,
        -2.72549802030452e-05, 0.0028846325205598, 0.0071533622456222,
        0.4032554792433688, -1.0616371912983589, -2.4438467307025813,
        -0.4764853536580197]], dtype=object)

In [83]:
test_env.idx2asset

{0: 'SPLRCT',
 1: 'SPLRCL',
 2: 'SPLRCM',
 3: 'SPLRCREC',
 4: 'SPLRCS',
 5: 'SPSY',
 6: 'SPNY',
 7: 'SPXHC',
 8: 'SPLRCD',
 9: 'SPLRCI',
 10: 'SPLRCU',
 11: 'cash'}

In [84]:
test_env.market_df.columns

Index(['Unnamed: 0', 'Date', 'SPX_close', 'SPLRCT_close', 'SPLRCL_close',
       'SPLRCM_close', 'SPLRCREC_close', 'SPLRCS_close', 'SPSY_close',
       'SPNY_close', 'SPXHC_close', 'SPLRCD_close', 'SPLRCI_close',
       'SPLRCU_close', 'VIX_close', 'SPX_R', 'SPX_log_r', 'SPLRCT_R',
       'SPLRCT_log_r', 'SPLRCL_R', 'SPLRCL_log_r', 'SPLRCM_R', 'SPLRCM_log_r',
       'SPLRCREC_R', 'SPLRCREC_log_r', 'SPLRCS_R', 'SPLRCS_log_r', 'SPSY_R',
       'SPSY_log_r', 'SPNY_R', 'SPNY_log_r', 'SPXHC_R', 'SPXHC_log_r',
       'SPLRCD_R', 'SPLRCD_log_r', 'SPLRCI_R', 'SPLRCI_log_r', 'SPLRCU_R',
       'SPLRCU_log_r', 'SPX_vol20', 'SPX_vol60', 'SPX_vol20_div_vol60',
       'SPX_vol20_normalized', 'SPX_vol20_div_vol60_normalized',
       'VIX_close_normalized'],
      dtype='object')

In [85]:
# # PPO를 쓰기 위한 multiprocess 환경 설정

# env_id = "CartPole-v1"
# num_cpu = 4  # Number of processes to use
# # Create the vectorized environment
# env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])